In [4]:
import hexathello.aiPlayers as aiPlayers
import hexathello.AutoPlayer as autoPlayer
import hexathello.Engine as engine
import hexathello.history as history
import hexathello.jable as jable
import hexathello.printing as printing

import numpy as np

from os import path
import os
import math
import tensorflow as tf

from os import listdir
from os.path import join, isfile


# -- Settings
game_size: int = 5
player_count: int = 2

In [5]:
matchup_dir = "data/history/matchups"
matchup_files = [f for f in listdir(matchup_dir) if f.endswith(".json")]
print(matchup_files)

agent_ids_v1 = [
    "1x720_total720_size5_players2_v1",
    "2x360_total720_size5_players2_v1",
    "3x240_total720_size5_players2_v1",
    "4x180_total720_size5_players2_v1",
    "5x144_total720_size5_players2_v1",
    "6x120_total720_size5_players2_v1",
    "7x102_total714_size5_players2_v1"
]

['all_matchups.json']


In [6]:

for ai_id_v1 in agent_ids_v1:
    print(f"\n Retraining agent: {ai_id_v1}")

    training_pov = []

    relevant_files = [f for f in matchup_files if ai_id_v1 in f]

    match_frames = []
    for fname in matchup_files:
        full_path = join(matchup_dir, fname)
        
        history_fromDisk: jable.JyFrame = jable.read_file(
            full_path
        )
    
        history_decoded: jable.JyFrame = history.history_fromInt(
            history_fromDisk
        )
    
        if len(history_decoded) < 20:
            print('skipping')
            continue
    
    
        povHistory: jable.JyFrame = history.povHistory_from_literalHistory(
            history_decoded
        )

        del history_decoded
        del history_fromDisk

        match_frames.append(povHistory)


        input_size: int = len(povHistory[0, 'board_state'])
        output_size: int = len(povHistory[0, 'player_action'])

    keras_path_v1 = join("data", "ai", "vary_dimensions", f"{ai_id_v1}.keras")
    if not path.isfile(keras_path_v1):
        print(f"Model not found at {keras_path_v1}")
        continue

    brain_model = tf.keras.models.load_model(keras_path_v1)
    ai_id_v2 = ai_id_v1.replace("v1", "v2")
    keras_path_v2 = join("data", "ai", "vary_dimensions", f"{ai_id_v2}.keras")
    os.makedirs(os.path.dirname(keras_path_v2), exist_ok=True)

    # Init updated agent
    ai_agent = aiPlayers.KerasHexAgent(
        size=game_size,
        player_count=player_count,
        brain=brain_model,
        player_id=None,
        ai_id=ai_id_v2
    )

    # Save only the improved model under the v2 name
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=keras_path_v2,
        monitor='loss',
        mode='min',
        save_best_only=True
    )

    training_pov_frame: jable.JyFrame = jable.JyFrame(training_pov)

    # Train with only relevant matches
    for povHistory in match_frames:
        ai_agent.train(
            game_history=povHistory,
            epochs=10,
            callbacks=[checkpoint_cb]
        )

    print(f"Retrained and saved model as {keras_path_v2}")



 Retraining agent: 1x720_total720_size5_players2_v1
Epoch 1/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.5867
Epoch 2/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.8248
Epoch 3/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6348
Epoch 4/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4585
Epoch 5/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3419
Epoch 6/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2544
Epoch 7/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2436
Epoch 8/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1510
Epoch 9/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1277
Epoch 10/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1202
Retrained and saved model as data\ai\vary_dimensions\1x720_total720_size5_players2_v2.keras

 Retraining agent: 2x360_total720_size5_players2_v1
Epoch 1/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.5182
Epoch 2/10
323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.99